## Week 10 and 11 Assignment - DATASCI200 Introduction to Data Science Programming, UC Berkeley MIDS

Write code in this Jupyter Notebook to solve the following problems. Please upload this **Notebook** with your solutions to your GitHub repository and provide a link in the last question in gradescope. 

Assignment due date: 11:59PM PT the night before the Week 12 Live Session. Do **NOT** push/upload the data file. 

## Objectives

- Explore and glean insights from a real dataset using pandas
- Practice using pandas for exploratory analysis, information gathering, and discovery
- Practice cleaning data and answering questions

## General Guidelines:

- This is a **real** dataset and so it may contain errors and other pecularities to work through
- This dataset is ~218mb, which will take some time to load (and probably won't load in Google Sheets or Excel)
- If you make assumptions, annotate them in your responses
- While there is one code/markdown cell positioned after each question as a placeholder, some of your code/responses may require multiple cells
- Double-click the markdown cells that say for example **1a answer here:** to enter your written answers. If you need more cells for your written answers, make them markdown cells (rather than code cells)
- This homework assignment is not autograded because of the variety of responses one could give. 
  - Please upload this notebook to the autograder page and the TAs will manually grade it. 
  - Ensure that each cell is run and outputs your answer for ease of grading! 
  - Highly suggest to do a `restart & run all` before uploading your code to ensure everything runs and outputs correctly.
  - Answers without code (or code that runs) will be given 0 points.
- **This is meant to simulate real world data so you will have to do some external research to determine what some of the answers are!** 

## Dataset

You are to analyze campaign contributions to the 2016 U.S. presidential primary races made in California. Use the csv file located here: https://drive.google.com/file/d/1Lgg-PwXQ6TQLDowd6XyBxZw5g1NGWPjB/view?usp=sharing. You should download and save this file in the same folder as this notebook is stored.  This file originally came from the U.S. Federal Election Commission (https://www.fec.gov/).

**DO NOT PUSH THIS FILE TO YOUR GITHUB REPO!**

- Best practice is to not have DATA files in your code repo. As shown below, the default load is outside of the folder this notebook is in. If you change the folder where the file is stored please update the first cell!
- If you do accidentally push the file to your github repo - follow the directions here to fix it: https://docs.google.com/document/d/15Irgb5V5G7pKPWgAerH7FPMpKeQRunbNflaW-hR2hTA/edit?usp=sharing

Documentation for this data can be found here: https://drive.google.com/file/d/11o_SByceenv0NgNMstM-dxC1jL7I9fHL/view?usp=sharing

## Data Questions

You are working for a California state-wide election campaign. Your boss wants you to examine historic 2016 election contribution data to see what zipcodes are more supportive of fundraising for your candidate. 

Your boss asks you to filter out some of the records:
- Only use primary 2016 contribution data (more like how your race is).
- Concentrate on Bernie Sanders as a candidate (most a like your candidate)

The questions your boss wants answered is:
- Which zipcode (5-digit zipcode) had the highest count of contributions and the most dollar amount?
- What day(s) of the month do most people donate?

## Setup

Run the cell below as it will load the data into a pandas dataframe named `contrib`. Note that a custom date parser is defined to speed up loading. If Python were to guess the date format, it would take even longer to load.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# These commands below set some options for pandas and to have matplotlib show the charts in the notebook
pd.set_option('display.max_rows', 1000)
pd.options.display.float_format = '{:,.2f}'.format

# Define a date parser to pass to read_csv
d = lambda x: datetime.strptime(x, '%d-%b-%y')

# Load the data
# We have this defaulted to the folder OUTSIDE of your repo - please change it as needed
contrib = pd.read_csv('../../../P00000001-CA.csv', index_col=False, parse_dates=['contb_receipt_dt'], date_parser=d)

# Note - for now, it is okay to ignore the warning about mixed types. 

/var/folders/_5/mw87wr4s5p902pvj5xm4sqnw0000gn/T/ipykernel_7841/3235627308.py:14: DtypeWarning: Columns (6,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  contrib = pd.read_csv('../../../P00000001-CA.csv', index_col=False, parse_dates=['contb_receipt_dt'], date_parser=d)


***
## 1. Initial Data Checks (50 points)

First we will take a preliminary look at the data to check that it was loaded correctly and contains the info we need.

The questions to answer at the end of this section:
- Do we have the correct # of columns and rows. 
- Do the records contain data for the questions we want to answer 
- What columns are important? 
- What columns can be dropped?
- What are the data problems?

---

**1a.** Print the *shape* of the data. Does this match the expectation? (2 points)

In [2]:
# 1a YOUR CODE HERE

# contrib.head(-10)

contrib.shape

(1125659, 18)

- **1a answer here:** 

The shape matches expectations. I used contrib.head(-10) to get a list of the last 10 rows and see that that last row is 1125659 and the count of columns is 18

---

**1b.** Print a list of column names. Are all the columns included that are in the documentation? (2 points)

In [3]:
# 1b YOUR CODE HERE

contrib.columns

Index(['cmte_id', 'cand_id', 'cand_nm', 'contbr_nm', 'contbr_city',
       'contbr_st', 'contbr_zip', 'contbr_employer', 'contbr_occupation',
       'contb_receipt_amt', 'contb_receipt_dt', 'receipt_desc', 'memo_cd',
       'memo_text', 'form_tp', 'file_num', 'tran_id', 'election_tp'],
      dtype='object')

- **1b answer here:** 

I reviewed several of the content in the 'download bulk data' section of the Federal Election Commission site. The primary file is  'Individual Contributions' which has the following columns. This matches with some of the content in P00000001-CA.csv but not all. 

******* list of fields in the individual contributions file ***********
https://www.fec.gov/data/receipts/individual-contributions

CMTE_ID
AMNDT_IND
RPT_TP
TRANSACTION_PGI
IMAGE_NUM
TRANSACTION_TP
ENTITY_TP
NAME
CITY
STATE
ZIP_CODE
EMPLOYER
OCCUPATION
TRANSACTION_DT
TRANSACTION_AMT
OTHER_ID
TRAN_ID	FILE_NUM
MEMO_CD
MEMO_TEXT
SUB_ID

********* I believe some of the data has been compiled from other files **************

I can't say for sure if the 18 columsns are precisely what was in the documentation as I could not find a single bulk file with the 18 fields. However, I reviewed several files (like those listed below) and reason to believe that the data is complete as it has been blended from multiple sources and the relevant information needed to answer the question is contained.

https://www.fec.gov/data/receipts/individual-contributions/?two_year_transaction_period=2016&contributor_state=CA&recipient_committee_type=P

https://www.fec.gov/data/browse-data/?tab=bulk-data

https://www.fec.gov/campaign-finance-data/candidate-committee-linkage-file-description/

https://www.fec.gov/campaign-finance-data/all-candidates-file-description/

https://www.fec.gov/campaign-finance-data/committee-master-file-description/

https://www.fec.gov/campaign-finance-data/contributions-committees-candidates-file-description/

---

**1c** Print out the first five rows of the dataset. How do the columns `cand_id`, `cand_nm` and `contbr_st` look? (3 points)

In [4]:
# 1c YOUR CODE HERE
contrib.head()

,cmte_id,cand_id,cand_nm,contbr_nm,contbr_city,contbr_st,contbr_zip,contbr_employer,contbr_occupation,contb_receipt_amt,contb_receipt_dt,receipt_desc,memo_cd,memo_text,form_tp,file_num,tran_id,election_tp
0,C00575795,P00003392,"Clinton, Hillary Rodham","AULL, ANNE",LARKSPUR,CA,"949,391,913.00",NaN,RETIRED,50.00,2016-04-26,NaN,X,* HILLARY VICTORY FUND,SA18,1091718,C4768722,P2016
1,C00575795,P00003392,"Clinton, Hillary Rodham","CARROLL, MARYJEAN",CAMBRIA,CA,"934,284,638.00",NaN,RETIRED,200.00,2016-04-20,NaN,X,* HILLARY VICTORY FUND,SA18,1091718,C4747242,P2016
2,C00575795,P00003392,"Clinton, Hillary Rodham","GANDARA, DESIREE",FONTANA,CA,"923,371,507.00",NaN,RETIRED,5.00,2016-04-02,NaN,X,* HILLARY VICTORY FUND,SA18,1091718,C4666603,P2016
3,C00577130,P60007168,"Sanders, Bernard","LEE, ALAN",CAMARILLO,CA,"930,111,214.00",AT&T GOVERNMENT SOLUTIONS,SOFTWARE ENGINEER,40.00,2016-03-04,NaN,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,SA17A,1077404,VPF7BKWA097,P2016
4,C00577130,P60007168,"Sanders, Bernard","LEONELLI, ODETTE",REDONDO BEACH,CA,"902,784,310.00",VERICOR ENTERPRISES INC.,PHARMACIST,35.00,2016-03-05,NaN,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,SA17A,1077404,VPF7BKX3MB3,P2016


- **1c answer here:** 

The fields are 

* cand_id (ie candidate id) which is a alphanumeric type. 
* The cand_nm (ie candidate name) is alphabetic (unless we have a name like X Æ A-Xii, thanks to Elon). 
* The contbd_st (constribution state) is a 2 digid alphabet (caps) that stands for CA (as this data is limited to california)

---

**1d.** Print out the values for the column `election_tp`. In your own words, based on the documentation, what information does the `election_tp` variable contain? Do the values in the column match the documentation? (3 points)

In [5]:
# 1d YOUR CODE HERE

contrib.election_tp.unique()

array(['P2016', 'G2016', nan, 'P2020'], dtype=object)

- **1d answer here:** 

I see 4 valuesP2016, G2016, P2020 and a nan). P2016 is for the 2016 Primary Season while the others are not relevant to the question on hand. P2020 is for 2020 Primaries while G2016 is for the 2016 General Eletion. Also, the nan values have to be fixed as well

---

**1e.** Print out the datatypes for all of the columns. What are the datatypes for the `contbr_zip`, `contb_receipt_amt`, `contb_receipt_dt`? (5 points)

In [6]:
# 1e YOUR CODE HERE

#type(contrib)

contrib.info()

#contrib.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1125659 entries, 0 to 1125658
Data columns (total 18 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   cmte_id            1125659 non-null  object        
 1   cand_id            1125659 non-null  object        
 2   cand_nm            1125659 non-null  object        
 3   contbr_nm          1125659 non-null  object        
 4   contbr_city        1125633 non-null  object        
 5   contbr_st          1125659 non-null  object        
 6   contbr_zip         1125564 non-null  object        
 7   contbr_employer    967757 non-null   object        
 8   contbr_occupation  1115260 non-null  object        
 9   contb_receipt_amt  1125659 non-null  float64       
 10  contb_receipt_dt   1125659 non-null  datetime64[ns]
 11  receipt_desc       15045 non-null    object        
 12  memo_cd            144268 non-null   object        
 13  memo_text          501148 n

- **1e answer here:** 

* contbr_zip : object
* contb_receipt_amt : float64
* contb_receipt_dt: datetime64[ns]


---

**1f.** What columns have the most non-nulls?  Would you recommend to drop any columns based on the number of nulls? (5 points)

In [7]:
# 1f YOUR CODE HERE
contrib.info()
#contrib.isnull().sum(axis = 0)

# to get the column with the least non-null values
#contrib.count().idxmin()

# to get the column with the most non-null values
contrib.count().idxmax()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1125659 entries, 0 to 1125658
Data columns (total 18 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   cmte_id            1125659 non-null  object        
 1   cand_id            1125659 non-null  object        
 2   cand_nm            1125659 non-null  object        
 3   contbr_nm          1125659 non-null  object        
 4   contbr_city        1125633 non-null  object        
 5   contbr_st          1125659 non-null  object        
 6   contbr_zip         1125564 non-null  object        
 7   contbr_employer    967757 non-null   object        
 8   contbr_occupation  1115260 non-null  object        
 9   contb_receipt_amt  1125659 non-null  float64       
 10  contb_receipt_dt   1125659 non-null  datetime64[ns]
 11  receipt_desc       15045 non-null    object        
 12  memo_cd            144268 non-null   object        
 13  memo_text          501148 n

'cmte_id'

- **1f answer here:** 

I am using multiple queries here -- starting with contrib.info() which gives me the detailed listing (refer 1g). I just validated if I am on the right track with query contrib.count().idxmax()

The max count (for non-null values) is 1125659 and there are a number of columns have the most non-null values. In order they are
* 0   cmte_id            1125659 non-null  object        
* 1   cand_id            1125659 non-null  object        
* 2   cand_nm            1125659 non-null  object        
* 3   contbr_nm          1125659 non-null  object
* 5   contbr_st          1125659 non-null  object 
* 9   contb_receipt_amt  1125659 non-null  float64       
* 10  contb_receipt_dt   1125659 non-null  datetime64[ns]
* 14  form_tp            1125659 non-null  object        
* 15  file_num           1125659 non-null  int64         
* 16  tran_id            1125659 non-null  object        

receipt_desc has the least number of non-null values (15045), implying there are 1110614, or ~ 99% of values are null. This is a candidate to be removed

I would also consider the following two columns for removal as well as they don't seem relevant to the basic question and have a ton of null values

Percentage of non-null values: 

* memo_cd: 87%
* memo_text: 55%

---

**1g.** A column we know that we want to use is the cand_nm column.  From the documentation each candidate is a unique candidate id also. Check data quality of `cand_id` column to see if it matches `cand_nm` column. Specifically check to ensure our targetted candidate 'Bernard Sanders' always has the same cand_id throughout. Any issues with `cand_nm` matching `cand_id`? (5 points)

In [2]:
# 1g YOUR CODE HERE

# this gives me a list of all records for Bernie
df1 = contrib.loc[contrib['cand_nm'].str.contains('Sanders, Bernard')]

# this is a lsit of all bernie records in California
df2 = df1.loc[df1['contbr_st']=='CA']

# get a list of all candidate ids for bernie in california
df2['cand_id'].unique()

array(['P60007168'], dtype=object)

- **1g answer here:** 

appears that bernie has candidate id of 'P60007168' throughout the CA dataset. No problem

---

**1h.** Another area to check is to make sure all of the records are from California. Check the `contbr_st` column - are there any records outside of California based on `contbr_st`? (5 points)

In [9]:
# 1h YOUR CODE HERE
contrib['contbr_st'].unique()

array(['CA'], dtype=object)

- **1h answer here:** 

it appears that all the records are from california. We do not see any records from outside California

---

**1i.** The next column to check for the analysis is the `tran_id` column. This column could be the primary key so look for duplicates. How many duplicate entries are there? Any pattern for why are there duplicate entries? (5 points)

In [3]:
# 1i YOUR CODE HERE

df_dupes = pd.concat(x for _, x in contrib.groupby("tran_id") if len(x) > 1)

len(df_dupes.index)

6873

---

- **1i answer here:** 

* yes, there are several duplicate entries (6873 to be precise)
* The interesting coincidence is that they seem to occur in 3's!
* There are others where the duplicates are in 2's where the only variable seems to be where the 
* -- contbr_occupation is 'Information Requested'

* These also are cases where the contributor zip code is incorrect. 

* There are other variations but these are the primary ones

---

**1j.** Another column to check is the `contb_receipt_amt` that shows the donation amounts. How many negative donations are included? What do negative donations mean? Please show at least pull a few rows to look at the records with negative donations. Do these records match with the expectation of why a negative donation would happen? (5 points)

In [4]:
# 1j YOUR CODE HERE

contrib_neg = contrib[contrib['contb_receipt_amt'] < 0]

neg_contributions = contrib_neg['contb_receipt_amt'].count()

print('the total number of negative contributions are :', neg_contributions)

contrib_neg[:10]

the total number of negative contributions are : 11896


,cmte_id,cand_id,cand_nm,contbr_nm,contbr_city,contbr_st,contbr_zip,contbr_employer,contbr_occupation,contb_receipt_amt,contb_receipt_dt,receipt_desc,memo_cd,memo_text,form_tp,file_num,tran_id,election_tp
19,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","JOLLIFF, RICHARD",CHICO,CA,"959,289,507.00",SELF EMPLOYED,RANCHER,-25.00,2016-04-29,REDESIGNATION TO GENERAL,X,REDESIGNATION TO GENERAL,SA17A,1077664,SA17A.1826482B,P2016
23,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","JOLLIFF, RICHARD",CHICO,CA,"959,289,507.00",SELF EMPLOYED,RANCHER,-150.00,2016-04-29,REDESIGNATION TO GENERAL,X,REDESIGNATION TO GENERAL,SA17A,1077664,SA17A.1826483B,P2016
81,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","JOLLIFF, RICHARD",CHICO,CA,"959,289,507.00",SELF EMPLOYED,RANCHER,-60.00,2016-04-14,REDESIGNATION TO GENERAL,X,REDESIGNATION TO GENERAL,SA17A,1077664,SA17A.1827494,P2016
190,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","NOWELL, DIANA L.",RANCHO SANTA MARGARITA,CA,"926,884,928.00",CAPISTRAND UNIFIED SCHOOL DISTRICT,LIBRARIAN TECHNICIAN,-100.00,2016-04-11,REDESIGNATION TO GENERAL,X,REDESIGNATION TO GENERAL,SA17A,1077664,SA17A.1639830B,P2016
213,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","LICHTY, ANDREW MR.",SAN DIEGO,CA,"921,096,720.00",SELF EMPLOYED,REAL ESTATE,-25.00,2016-04-30,REDESIGNATION TO GENERAL,X,REDESIGNATION TO GENERAL,SA17A,1077664,SA17A.1826888B,P2016
220,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","LICHTY, ANDREW MR.",SAN DIEGO,CA,"921,096,720.00",SELF EMPLOYED,REAL ESTATE,-25.00,2016-04-30,REDESIGNATION TO GENERAL,X,REDESIGNATION TO GENERAL,SA17A,1077664,SA17A.1826894B,P2016
223,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","LICHTY, ANDREW MR.",SAN DIEGO,CA,"921,096,720.00",SELF EMPLOYED,REAL ESTATE,-50.00,2016-04-30,REDESIGNATION TO GENERAL,X,REDESIGNATION TO GENERAL,SA17A,1077664,SA17A.1826895B,P2016
231,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","LICHTY, ANDREW MR.",SAN DIEGO,CA,"921,096,720.00",SELF EMPLOYED,REAL ESTATE,-50.00,2016-04-30,REDESIGNATION TO GENERAL,X,REDESIGNATION TO GENERAL,SA17A,1077664,SA17A.1826886B,P2016
261,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","NOWELL, DIANA L.",RANCHO SANTA MARGARITA,CA,"926,884,928.00",CAPISTRAND UNIFIED SCHOOL DISTRICT,LIBRARIAN TECHNICIAN,-150.00,2016-04-11,REDESIGNATION TO GENERAL,X,REDESIGNATION TO GENERAL,SA17A,1077664,SA17A.1639829B,P2016
307,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","LICHTY, ANDREW MR.",SAN DIEGO,CA,"921,096,720.00",SELF EMPLOYED,REAL ESTATE,-50.00,2016-04-30,REDESIGNATION TO GENERAL,X,REDESIGNATION TO GENERAL,SA17A,1077664,SA17A.1826889B,P2016


- **1j answer here:**

NOTE: The question does not specify if the scope is for P2016 only. I have analyzed the full dataset unless the questions specifically states, 2016 primary season

* total number of positive values = 1113763

* total number of negative values = 11896

* sum of counts of negative and positive values = 1125659 (which is the total number of records)

* about 1% of thse records have a -ve contb_receipt_amt value

* it appears that the -ve values are driven by the fact that this is an accounting transaction where these amounnts are moved to a different account. You can see this by the 'receipt_desc' as 'REDESIGNATION TO GENERAL' or other reasons like 'REATTRIBUTION TO SPOUSE'. The full list of these account related charges are shown below

'REDESIGNATION TO GENERAL', nan, '* HILLARY VICTORY FUND',
       'REATTRIBUTION TO SPOUSE', 'CHARGED BACK',
       'REDESIGNATION TO PRESIDENTIAL GENERAL',
       'TRANSFER FROM RUBIO VICTORY',
       '2016 SENATE PRIMARY DONOR REDESIGNATION TO GENERAL',
       'REDESIGNATION FROM PRIMARY',
       'US CITIZEN REDESIGNATION TO GENERAL',
       'REDESIGNATION TO CRUZ FOR SENATE',
       '* REATTRIBUTED TO JOHN CASTOR', 'NSF/RETURNED CHECK',
       '* REATTRIBUTED TO FRANK DARABONT',
       'REFUND TO BE ISSUED; REDESIGNATION TO GENERAL',
       'REFUND TO BE ISSUED; CHARGED BACK',
       'REATTRIBUTION TO SPOUSE; SEE REDESIGNATION',
       'CHECK RETURNED BY BANK', 'OVERPAYMENT', '*BEST EFFORTS UPDATE',
       '* REATTRIBUTED TO CHRISTINE MARSHALL',
       '* REATTRIBUTED TO CAROL FITZMORRIS',
       '* REATTRIBUTED TO ERIC UHRANE', '* REATTRIBUTED TO YEU-WEI YEE',
       '* REATTRIBUTED TO NORAH STONE', '* REATTRIBUTED FROM KIM RIDDER',
       'INSUFFICIENT FUNDS', 'REDESIGNATION TO PRIMARY',
       'REATTRIBUTION TO SPOUSE, TERRIE SCHULTZ.',
       '* HILLARY ACTION FUND', 'VOIDED 9/16/16', '2016 SENATE GENERAL',
       '* REATTRIBUTION TO SPOUSE, TERRIE SCHULTZ.', 'REISSUED ON 6/21',
       'CHARGEBACK REVERSAL', 'REATTRIBUTION TO SPOUSE, HARRY HICKS.',
       'REATTRIBUTION/REDESIGNATION REQUESTED REDESIGNATION TO GENERAL',
       '* REATTRIBUTED TO ELIZABETH DOYLE',
       '* REATTRIBUTED TO PRUDENCE MOORE',
       '* REATTRIBUTED TO AISHA LYNCH',
       '* REATTRIBUTED TO SHEILA NEWBERRY',
       '* REATTRIBUTED TO JESSIKA CROIZAT',
       '* REATTRIBUTED TO JUDITH HAYNIE'], dtype=object)

---

**1k.** One more column to look at is the date of donation column. Are there any dates outside of the primary period (defined as 1 Jan 2014 to 7 June 2016)? Are the dates well-formatted for our analysis? (5 points)

In [5]:
# 1k YOUR CODE HERE

# contrib['election_tp'].unique()

# 1 first filter out the non P2016 records and save contents in a new df

contrib_p2016 = contrib[contrib['election_tp'] == 'P2016']

# 2 select records based on the data parameters

inscope_p2016 = contrib_p2016[(contrib_p2016['contb_receipt_dt'] >= '2014-01-01')  & \
                       (contrib_p2016['contb_receipt_dt'] <= '2016-06-07')]

# 3 count of records in scope
#len(inscope_p2016.index)
# 668864
p2016_valid = len(inscope_p2016.index)


# 4 count of records out of scope (from full dataframe)
# 1125659
contrib_full = len(contrib.index)

# 5 count of records of only the P2016 primary
#len(contrib_p2016.index)
# 810481
p2016_all = len(contrib_p2016.index)

# 6 calculate the invalid records

print("invalid records in 2016 primary onlyis : ", p2016_all - p2016_valid)

invalid records in 2016 primary onlyis :  141617


- **1k answer here:**
* first, isolcate the records in the contrib df to only those for the primary campaign 2016. I see the file contains data from 3 cycles ['P2016', 'G2016', nan, 'P2020']. So, we need to exclude the others first. This may be contentious and this step may not be needed because the label may be misclassified, One can argue that we could start with step 2 directly. 

* the date of the donation is the field 'receipt_dt'
* as you can see from above, I have taken 5 steps (admittedly this can be done in a single line, but that would be 
  hard to interpret as yet)
* I first calculate the number of inscope rows (from the already narrowed df of contrib_p2016)
* I then compare that with the full list of rows in the p2016 section
* the final answer is 141617 records that are outside the date range

---

**1l.** Finally, answer the initial questions in the cells below (5 points)

---

**1l.1** Do we have the correct # of columns and rows.

- **1l.1 answer here:**

Not sure what this exactly means. If by 'correct', you mean 'adequate', the answer is yes we do have the adequate data in order to answer the two questions. Having 407172 records for Bernie is a good enouigh sample size for us to work with. It also shows that Bernie got 36% of all contributions, which is pretty cool.

bernie = len(contrib[contrib['cand_nm'].str.contains('Sanders, Bernard')].index)
total = len(contrib.index)
print(bernie/total)

---

**1l.2** Do the records contain data for the questions we want to answer?

- **1l.2 answer here:**

The two primary questions are about the most supportive zip code (5 digit) and the days of the month are people most likely to donate. However we have a problem

The zipcode field is not really 5 digits -- it is more like the expanded 9 digit format. Infact, only 2% of the records are 5 digits and the rest are greater than 5. More research needs to be done for the > 5 digit data and clean it, which could be as simply as taking just the first 5 digits and ignoring the rest.

------------------------ query used ------------------------
The query I used was
zip_greater_5 = len(contrib[contrib['contbr_zip'].str.len() > 5].index)
zip_5 = len(contrib[contrib['contbr_zip'].str.len() == 5].index)

print(zip_5/zip_greater_5)

Coming to the dates, it appears that all records are of length 10, which is good. This means, we don't have records less than 10 or greater than 10. However, further research needs to be done if the order (yyyy-mm-dd) is consistent across all records

------------------------ query used ------------------------
date_string = contrib['contb_receipt_dt'].astype('str')
date_string_10 = date_string.str.len() == 10
date_string_10.value_counts()
#type(date_string_10)
#len(date_string_10)
date_string_less_10 = date_string.str.len() < 10
date_string_less_10.value_counts()
date_string_greater_10 = date_string.str.len() > 10

---

**1l.3** What columns are important?

- **1l.3 answer here:** 

The two questions are (1) Which zipcode (5-digit zipcode) had the highest count of contributions and the most dollar amount? and (2) What day(s) of the month do most people donate?

to answer these 2 questions, the following 5 fields are most relevant

--- filter first ---
* cand_nm (filtered for only Bernie)
* election_to (filtered for only P2016)

--- key data -----
* contr_zip
* contb_receipt_amt
* contb_receipt_dt


---

**1l.4** What columns can be dropped?

- **1l.4 answer here:** 

The following columns can be dropped

* cmte_id'
* 'contbr_nm', 
* 'contbr_city',
* 'contbr_st', 
* 'contbr_employer', 
* 'contbr_occupation',  
* 'form_tp', 
* 'file_num', 
* 'tran_id',

While not entirely relevant to the question, I would like to retain the following fields as they help clarify accounting discrepencies
* 'receipt_desc', 
* 'memo_cd',
* 'memo_text',

---

**1l.5** What are the data problems?

- **1l.5 answer here:**

There are several data problems, but the primary one being the following
* -ve receipt amounts
* duplicate records (which in cases is tired to the -ve receipt amounts as they are accounting transfers / corrections)

Some minor data issues include
* receipt_desc is NaN
* memo_cd contains irrelevant / incorrect / Nan

On a side note, I was concerned if the candidate name was missplet in the data, but no. I valided this with the query

* bernie_all = contrib[contrib['cand_nm'].str.contains('Bern')]
* bernie_all['cand_nm'].unique()

---

**1l.6** List any assumptions so far:

- **1l.6 answer here:**

* All the contribtions are from either american citizens or those resident here. Foreign contributions are excluded 
* All contributions were done legally and following campaign contribution guidelines
* Individual contributions are only done up to the max allowed ($1000 per candidate per election)
* Contributions are limited to only those for a candidate. Contribution to Super-PACs or political parties is excluded.
* Fields like receipt_desc have many null values. This is concerning but assumed to be benign
* Memo_cd in particular has many null or irrelevant values. This is also assumed to be benigh
* All contributions are in US dollars
* Tran_id has multiple formats, which perhaps means it originated from multiple places. Assumption here that this value is benign and does not influence the outcome of the analysis

***
## 2. Data filtering and data quality fixes (30 points)

Now that we have a basic understanding of the data, let's filter out the records we don't need and fix the data.

---

**2a.** From the dataset filter out (remove) any election_tp not in the primary election. Print/show the shape of the dataframe after the filtering is complete. (5 points)

In [13]:
# 2a YOUR CODE HERE

contrib_p2016 = contrib[contrib['election_tp'] == 'P2016']

contrib_shape = contrib_p2016.shape
print('Contribution shape after filtering only for 2016 CA primary is :', contrib_shape)
#print(contrib_p2016.shape())
#contrib_p2016

Contribution shape after filtering only for 2016 CA primary is : (810481, 18)


---

**2b.** From the dataset filter out (remove) any candidate that is not Bernie Sanders. Print/show the shape of the dataframe after the filtering is complete. (5 points)

In [27]:
# 2b YOUR CODE HERE

## assumption -- I am assuming 2a as sequential to 2b adn beyond. for example, this means that the analysis
## is limited to 2016 primary elections only


# start by confirming all of bernie records
#bernie_all = contrib[contrib['cand_nm'].str.contains('Bern')]

bernie_2016p = contrib_p2016[contrib_p2016['cand_nm'].str.contains('Bern')]

# I chose a .contains('Bern') in order to check if there are records with the name misspelt
#bernie_2016p['cand_nm'].unique()
# the outcome from this confirms there is just one candidate record for 'Sanders, Bernard'
#bernie

# print shape of bernie only records for P2016

print('Shape of Bernie-only data for 2016 CA Primaries is ', bernie_2016p.shape)

Shape of Bernie-only data for 2016 CA Primaries is  (407171, 18)


**2c.** The `contbr_zip` column is not formatted well for our analysis. Make a new zipcode column that is the five-digit zipcodes. Filter out any records outside of California based on the zipcode. Print/show the shape of the dataframe after the filtering is complete. (10 points).

- You will have to research what the valid 5-digit zipcodes for California are!

---

In [31]:
# 2c YOUR CODE HERE

#I have followed a multi-step process to (1) extract the first 5 numbers in the zip code and then to filter them 
# from a list of california zip codes

#1 import zipcodes library
import zipcodes

#2 extract first 5 characters from the zip code field. Store as 'temp' field

bernie_2016p['temp']  = bernie_2016p['contbr_zip'].astype(str).str[:5]
#bernie_2016p_temp = bernie_2016p['contbr_zip'].astype(str).str[:5]
#bernie_2016p_temp

#3 convert the new field (temp) to an integer

bernie_2016p['zipcode_5'] = pd.to_numeric(bernie_2016p['temp'], errors='coerce').fillna(0).astype(np.int64)
#bernie_2016p

#4 get a list of all zipodes in California and store as a list 'res'

filtered_zips = zipcodes.filter_by(state="CA")
#filtered_zips

res = [sub['zip_code'] for sub in filtered_zips]

#5 convert res (string) to res_int(integers) 
res_int = [eval(i) for i in res]
#res_int

#6 filter the dataframe for only california zip codes

#bernie_2016p[bernie_2016p['zipcode_5'].isin(res_int)]
#bernie_2016p_cazip = bernie_2016p[bernie_2016p['zipcode_5'].isin(res_int)]


#print('The shape of the new dataframe is ', bernie_2016p_cazip.shape)
print("The shape of the data is ", bernie_2016p[bernie_2016p['zipcode_5'].isin(res_int)].shape)

/var/folders/_5/mw87wr4s5p902pvj5xm4sqnw0000gn/T/ipykernel_18637/1091508623.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bernie_2016p['temp']  = bernie_2016p['contbr_zip'].astype(str).str[:5]
/var/folders/_5/mw87wr4s5p902pvj5xm4sqnw0000gn/T/ipykernel_18637/1091508623.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bernie_2016p['zipcode_5'] = pd.to_numeric(bernie_2016p['temp'], errors='coerce').fillna(0).astype(np.int64)


The shape of the data is  (407085, 20)


---

**2d.** The receipt amount column has negative donations. After talking with your team, a decision was made that the best course of action is to remove these negative values so that the donation count and amount is more accurate. Print/show the shape of the dataframe after the filtering is complete. (5 points)

In [33]:
# 2d YOUR CODE HERE

#1 start by creating a new dataframe (I personally don't like to pollute existing dataframe or overwrite them)
###### I understand this isn't the smartest way to code, but I have my preferences for ease of recollection adn reuse
bernie_2016p_CA = bernie_2016p[bernie_2016p['zipcode_5'].isin(res_int)]
bernie_2016p_CA

#2 identify the records with a negative contribution amount

bernie_2016p_CA[bernie_2016p_CA['contb_receipt_amt']<0]

#3 Drop these records from the bernie_2016p_CA df. I have chosen to save this in a new dataframe with just +ve values

bernie_2016p_CA_positive = bernie_2016p_CA.drop(bernie_2016p_CA[bernie_2016p_CA['contb_receipt_amt']<0].index)
#bernie_2016p_CA_positive.shape

#4 you will notice 20 columns (instead). Drop the 2 columns
#bernie_2016p_CA_positive = bernie_2016p_CA_positive.drop('temp', axis = 1)
#bernie_2016p_CA_positive = bernie_2016p_CA_positive.drop('zipcode_5', axis = 1)

#5 get shape of the data

bernie_2016p_CA_positive.shape


(403998, 20)

---

**2e.** From the dataset drop any columns that won't be used in the analysis. Print/show the shape of the dataframe after the dropping is complete. What columns did you drop and why? (5 points)

In [17]:
# 2e YOUR CODE HERE
# bernie_2016p_CA_positive.columns

bernie_2016p_CA_positive.drop(['cmte_id', 'contbr_zip', 'cand_nm', 'cand_id', 'contbr_nm', 'contbr_city','contbr_st',\
                             'contbr_employer', 'contbr_occupation','receipt_desc', 'memo_cd',\
                             'memo_text', 'form_tp', 'file_num', 'tran_id', 'election_tp', 'temp'], \
                              axis = 1, inplace = True)
print('The final dataframe for analysis has 3 columns only ', bernie_2016p_CA_positive.shape)

The final dataframe for analysis has 3 columns only  (403998, 3)


- **2e answer here:**

I have retained only 3 columns (contb_receipt_amt, contb_receipt_dt,zipcode_5 ) and dropped the rest.

I believe these are the only columns needed to answer the 2 questions on hand

---

**2f.** List any assumptions that you made up to this point:

- **2f answer here:**

* I have dropped several fields that could be interested from an analytical point of view, but not relevant for the question on hand. These fields are: 'cmte_id', 'contbr_zip', 'cand_nm', 'cand_id', 'contbr_nm', 'contbr_city','contbr_st','contbr_employer', 'contbr_occupation','receipt_desc', 'memo_cd', 'memo_text', 'form_tp', 'file_num', 'tran_id', 'election_tp', 'temp'
* so the primary assumption is regarding the scope of the analysis which is quite limited. 
* I got the list of california zip codes from an external library and believe this to be accurate
* all contribution amounts are in us dollars

***
## 3. Answering the questions (20 points)

Now that the data is cleaned and filterd - let's answer the two questions from your boss!

**3a.** Which zipcode had the highest count of contributions and the most dollar amount? (10 points)

In [35]:
# 3a YOUR CODE HERE

################## PART 1: most prolific zipcode ##################
items_counts = bernie_2016p_CA_positive['zipcode_5'].value_counts(sort=False)
top = items_counts.loc[[items_counts.idxmax()]]
# value, count = top.index[0], top.iat[0]
# print(value, count)

print('the zip code with the highest count of contributions is :', top.index[0])

################## PART 2: max sum of contributions  ##################

max_sum = bernie_2016p_CA_positive.groupby('zipcode_5').agg({'contb_receipt_amt': ['sum']}).max()
print("the max sum collected is :", max_sum)

# max contribution amount zipcode
max_sum_zip = bernie_2016p_CA_positive.groupby('zipcode_5').agg({'contb_receipt_amt': ['sum']}).idxmax()
print("zipcode for the max sum :", max_sum_zip)

the zip code with the highest count of contributions is : 94110
the max sum collected is : contb_receipt_amt  sum   294,061.13
dtype: float64
zipcode for the max sum : contb_receipt_amt  sum    94110
dtype: int64


---

- **3a answer here:** 

I have two separate parts to the answer
* Part 1:  most prolific zipcode (ie with the highest count of contributions)
* the zipcode is 94110 (which ironically is the zipcode of where I lived till recently in San Francisco)


* Part 2: max sum of contributions  (this is the zipcode with the highest sum of contributions)
* the max sum collected is 294,061.13
* the zipcode where the max sum of contributions is collected is 94110


---

**3b.** What day(s) of the month do most people donate? (10 points)

In [19]:
# 3b YOUR CODE HERE

############## Based on contribution amount ############################

max_sum_date = bernie_2016p_CA_positive.groupby('contb_receipt_dt').agg({'contb_receipt_amt': ['sum']}).idxmax()
print("the max sum is collected on date :", max_sum_date)

max_sum_date_value = bernie_2016p_CA_positive.groupby('contb_receipt_dt').agg({'contb_receipt_amt': ['sum']}).max()
print("the max sum collected is :", max_sum_date_value)

############## Based on count of contributions ############################
#----------------------------------------------------------------------------------------
date_counts = bernie_2016p_CA_positive['contb_receipt_dt'].value_counts(sort=False)
top_date = date_counts.loc[[date_counts.idxmax()]]
# value, count = top.index[0], top.iat[0]
# print(value, count)

print('the most prolific date is :', top_date.index[0])
#date_counts
#.sort(ascending=False)
type(date_counts)

top_10_dates = date_counts.sort_values(ascending = False).head(10)
print('based on the count of contributions, the most likely dates for seeking contributions is :',top_10_dates)

the max sum is collected on date : contb_receipt_amt  sum   2016-02-29
dtype: datetime64[ns]
the max sum collected is : contb_receipt_amt  sum   460,480.99
dtype: float64
the most prolific date is : 2016-02-29 00:00:00
based on the count of contributions, the most likely dates for seeking contributions is : 2016-02-29    8458
2016-03-31    8436
2016-05-31    7084
2016-04-30    6689
2016-03-09    6494
2016-03-14    6396
2016-03-30    6389
2016-04-08    5611
2016-03-16    4898
2016-04-06    4822
Name: contb_receipt_dt, dtype: int64


- **3b answer here:** 

The question is a bit ambiguous and I have approached it in two ways

1. based on the sum of contributions; ie date with the highest total sum of contributions
-------- for this metric, the date with the highest sum of contributions is 2016-02-29 wiht a sum of 460,480.99

2. based on the count/number of occurances of contributions
-------- for this metric, I have listed the top 10 dates with a descending list of the count of contributers 
* based on this the most likely dates are towards the end of the month. As can be seen from the data, the most likely date is actually the 31st of the month followed by the 30th of the month. 
* so, it is fair to say that people are more likely to contribute towards the end of the month


---

## If you have feedback for this homework, please submit it using the link below:

http://goo.gl/forms/74yCiQTf6k